# 📊 Projet 11 – Étude de marché internationale pour *La Poule qui Chante*

## 🐔 Contexte
*La Poule qui Chante* est une entreprise française spécialisée dans l’élevage et la vente de poulets bio, sous le label “Poulet Agriculture Biologique”.

Actuellement active uniquement en France, elle souhaite évaluer les **opportunités d’expansion à l’international**.

## 🎯 Objectif de l’analyse
Identifier, à l’aide d’une analyse de données multi-sources :
- des **groupes de pays** aux profils similaires,
- afin de recommander les **zones géographiques prioritaires** pour l’exportation.

L’approche inclura :
- la **collecte de données agro-économiques et sociales**,
- une **réduction de dimensions par ACP**,
- une **segmentation via clustering (CAH et K-means)**.

## 🧑‍💻 Ce notebook
Ce document correspond à la **première phase** du projet :
> **Préparation et exploration des données**  
> (avant l’analyse ACP + clustering dans un second notebook)

In [2]:
# 📦 Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 1. 📁 Importation des données

Nous commençons par charger les deux fichiers fournis :
- `DisponibiliteAlimentaire_2017.csv` : disponibilité des aliments par pays (source : FAO)
- `Population_2000_2018.csv` : population totale par pays sur plusieurs années

In [4]:
# 📁 Chargement des fichiers CSV
df_dispo = pd.read_csv("DisponibiliteAlimentaire_2017.csv", encoding='utf-8')
df_pop = pd.read_csv("Population_2000_2018.csv", encoding='utf-8')

# Aperçu rapide
df_dispo.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.0,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.0,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.0,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.0,S,Données standardisées


### 2. 🔍 Exploration de la disponibilité alimentaire

Nous explorons le fichier pour identifier les éléments et produits pertinents à extraire, notamment la viande de volaille.

In [6]:
# Quels sont les produits disponibles ?
df_dispo['Produit'].unique()

array(['Blé et produits', 'Riz et produits', 'Orge et produits',
       'Maïs et produits', 'Seigle et produits', 'Avoine',
       'Millet et produits', 'Sorgho et produits', 'Céréales, Autres',
       'Pommes de Terre et produits', 'Ignames', 'Racines nda',
       'Sucre, canne', 'Sucre, betterave', 'Sucre Eq Brut',
       'Edulcorants Autres', 'Miel', 'Haricots', 'Pois',
       'Légumineuses Autres et produits', 'Noix et produits', 'Soja',
       'Arachides Decortiquees', 'Graines de tournesol',
       'Graines Colza/Moutarde', 'Graines de coton', 'Coco (Incl Coprah)',
       'Sésame', 'Olives', 'Plantes Oleiferes, Autre', 'Huile de Soja',
       "Huile d'Arachide", 'Huile de Tournesol',
       'Huile de Colza&Moutarde', 'Huile Graines de Coton',
       'Huile de Palmistes', 'Huile de Palme', 'Huile de Coco',
       'Huile de Sésame', "Huile d'Olive", 'Huile de Son de Riz',
       'Huile de Germe de Maïs', 'Huil Plantes Oleif Autr',
       'Tomates et produits', 'Oignons', 'Légumes, 

Le libellé qui nous intéresse s'écrit 'Viande de Volailles'

In [8]:
df_dispo['Élément'].unique()

array(['Production', 'Importations - Quantité', 'Variation de stock',
       'Exportations - Quantité', 'Disponibilité intérieure',
       'Aliments pour animaux', 'Semences', 'Pertes', 'Résidus',
       'Nourriture',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité de protéines en quantité (g/personne/jour)',
       'Disponibilité de matière grasse en quantité (g/personne/jour)',
       'Traitement', 'Autres utilisations (non alimentaire)',
       'Alimentation pour touristes'], dtype=object)

### 2.1 🐓 Extraction des variables FAO – Produit "Viande de Volailles"

Nous extrayons ici les 5 variables disponibles dans le fichier `DisponibiliteAlimentaire_2017.csv`, pour l’année 2017 et le produit **Viande de Volailles** :

- Production totale (en tonnes)
- Importations (en tonnes)
- Exportations (en tonnes)
- Disponibilité intérieure (en tonnes)
- Consommation par habitant (kg/hab/an)

Cela permettra d’analyser à la fois la **demande** (consommation), l’**offre locale** (production), et la **balance commerciale** (import/export) pour chaque pays.

In [10]:
# Filtrer par année et produit
annee = 2017
produit = "Viande de Volailles"

# Consommation par habitant (kg/personne/an) → kg/hab
conso = df_dispo[
    (df_dispo["Année"] == annee) &
    (df_dispo["Produit"] == produit) &
    (df_dispo["Élément"] == "Disponibilité alimentaire en quantité (kg/personne/an)")
][["Zone", "Valeur"]].rename(columns={"Valeur": "conso_volaille_kg_hab"})

# Production (milliers de tonnes)
prod = df_dispo[
    (df_dispo["Année"] == annee) &
    (df_dispo["Produit"] == produit) &
    (df_dispo["Élément"] == "Production")
][["Zone", "Valeur"]].rename(columns={"Valeur": "prod_volaille_milliers_tonnes"})

# Importations (milliers de tonnes)
imp = df_dispo[
    (df_dispo["Année"] == annee) &
    (df_dispo["Produit"] == produit) &
    (df_dispo["Élément"] == "Importations - Quantité")
][["Zone", "Valeur"]].rename(columns={"Valeur": "import_volaille_milliers_tonnes"})

# Exportations (milliers de tonnes)
exp = df_dispo[
    (df_dispo["Année"] == annee) &
    (df_dispo["Produit"] == produit) &
    (df_dispo["Élément"] == "Exportations - Quantité")
][["Zone", "Valeur"]].rename(columns={"Valeur": "export_volaille_milliers_tonnes"})

# Disponibilité intérieure (milliers de tonnes)
dispo = df_dispo[
    (df_dispo["Année"] == annee) &
    (df_dispo["Produit"] == produit) &
    (df_dispo["Élément"] == "Disponibilité intérieure")
][["Zone", "Valeur"]].rename(columns={"Valeur": "dispo_volaille_milliers_tonnes"})

# Variation de stock (milliers de tonnes)
var_stock = df_dispo[
    (df_dispo["Année"] == annee) &
    (df_dispo["Produit"] == produit) &
    (df_dispo["Élément"] == "Variation de stock")
][["Zone", "Valeur"]].rename(columns={"Valeur": "variation_stock_milliers_tonnes"})

# Fusion progressive
df_volaille = conso.merge(prod, on="Zone", how="outer") \
                   .merge(imp, on="Zone", how="outer") \
                   .merge(exp, on="Zone", how="outer") \
                   .merge(dispo, on="Zone", how="outer") \
                   .merge(var_stock, on="Zone", how="outer") \
                   .rename(columns={"Zone": "Pays"})

# Aperçu
df_volaille.head()

,Pays,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes
0,Afghanistan,1.53,28.0,29.0,NaN,57.0,0.0
1,Afrique du Sud,35.69,1667.0,514.0,63.0,2118.0,-0.0
2,Albanie,16.36,13.0,38.0,0.0,47.0,4.0
3,Algérie,6.38,275.0,2.0,0.0,277.0,0.0
4,Allemagne,19.47,1514.0,842.0,646.0,1739.0,-29.0


### 3. 👥 Extraction de la population totale par pays (2017)

Nous allons extraire la population totale pour chaque pays à une date proche de 2017, afin de pouvoir estimer la consommation globale de volaille par pays.

In [12]:
# 🔍 Exploration du fichier population
df_pop.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,20779.953,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,21606.988,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2002,2002,1000 personnes,22600.770,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2003,2003,1000 personnes,23680.871,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,24726.684,X,Sources internationales sûres,NaN


In [13]:
# Renommer la colonne "Zone" en "Pays" dans df_volaille
df_volaille.rename(columns={"Zone": "Pays"}, inplace=True)

### 3.1 🌍 Filtrage de la population en 2017

Nous extrayons ici la population totale pour chaque pays pour l'année 2017.  
Les données sont exprimées en milliers de personnes et seront ensuite utilisées pour estimer la consommation globale de volaille par pays.

In [15]:
# Filtrage de la population pour l’année 2017
df_pop_2017 = df_pop[df_pop['Année'] == 2017]

# Garde seulement les colonnes utiles
df_pop_2017 = df_pop_2017[['Zone', 'Valeur']]
df_pop_2017 = df_pop_2017.rename(columns={
    'Zone': 'Pays',
    'Valeur': 'Population_milliers'
})

# Aperçu
df_pop_2017.head()

,Pays,Population_milliers
17,Afghanistan,36296.113
36,Afrique du Sud,57009.756
55,Albanie,2884.169
74,Algérie,41389.189
93,Allemagne,82658.409


### 4. 🔗 Fusion des données consommation & population

Nous fusionnons les deux sources pour obtenir un tableau complet par pays.  
Nous en profitons pour estimer la consommation totale de viande de volaille par pays (en tonnes), en multipliant la consommation par habitant par la population.

In [17]:
# Jointure OUTER avec indicateur
df_check_pop = pd.merge(df_volaille, df_pop_2017, on='Pays', how='outer', indicator='source_jointure')

# Aperçu du résultat
df_check_pop['source_jointure'].value_counts()

source_jointure
both          172
right_only     64
left_only       0
Name: count, dtype: int64

In [18]:
# Afficher les pays qui n'ont pas matché
df_check_pop[df_check_pop['source_jointure'] != 'both'][['Pays', 'source_jointure']].sort_values(by='source_jointure')

,Pays,source_jointure
172,Andorre,right_only
206,Mayotte,right_only
207,Micronésie (États fédérés de),right_only
208,Monaco,right_only
209,Montserrat,right_only
...,...,...
199,Îles Turques-et-Caïques,right_only
200,Îles Vierges américaines,right_only
201,Îles Vierges britanniques,right_only
234,Tonga,right_only


### Fusion entre données FAO volaille et population

Nous réalisons ici une jointure entre les données de volaille (issues de la FAO) et les données de population (issues de la Banque mondiale).

Nous utilisons `how="left"` car nous choisissons de **conserver uniquement les pays pour lesquels nous avons des données sur la volaille**, notre base d’analyse principale.  
Les pays présents dans les autres fichiers (ex. population) mais absents des données volaille sont considérés comme **hors périmètre** et ne sont donc pas intégrés à l’analyse.

In [20]:
# Jointure finale avec la population
df_final = pd.merge(df_volaille, df_pop_2017, on='Pays', how='left')

# Aperçu
df_final.head()

,Pays,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers
0,Afghanistan,1.53,28.0,29.0,NaN,57.0,0.0,36296.113
1,Afrique du Sud,35.69,1667.0,514.0,63.0,2118.0,-0.0,57009.756
2,Albanie,16.36,13.0,38.0,0.0,47.0,4.0,2884.169
3,Algérie,6.38,275.0,2.0,0.0,277.0,0.0,41389.189
4,Allemagne,19.47,1514.0,842.0,646.0,1739.0,-29.0,82658.409


## 🔢 Variables sélectionnées pour l'étude de marché

| Nom de la variable             | Description                                                    | Source / Méthode d’extraction                                                            | Dimension PESTEL       |
|-------------------------------|----------------------------------------------------------------|-------------------------------------------------------------------------------------------|------------------------|
| `conso_volaille_kg_hab`       | Consommation de volaille en kg par habitant                    | FAO – Disponibilité alimentaire (`Élément = Disponibilité alimentaire (Kcal/hab/jour)`)  | Économique             |
| `prod_volaille_milliers_tonnes`        | Production de volaille en tonnes                               | FAO – Disponibilité alimentaire (`Élément = Production`)                                 | Économique             |
| `import_volaille_milliers_tonnes`      | Importations de volaille en tonnes                             | FAO – Disponibilité alimentaire (`Élément = Importations - Quantité`)                   | Économique             |
| `export_volaille_milliers_tonnes`      | Exportations de volaille en tonnes                             | FAO – Disponibilité alimentaire (`Élément = Exportations - Quantité`)                   | Économique             |
| `dispo_volaille_milliers_tonnes`       | Volaille disponible pour consommation (en tonnes)              | FAO – Disponibilité alimentaire (`Élément = Disponibilité alimentaire`)                 | Économique             |
| `Population_milliers`         | Population totale en milliers                                  | Banque Mondiale – fichier population                                                     | Démographique          |
| `PIB_hab_USD`                 | PIB par habitant en $US courants                               | Banque Mondiale – `PIB par habitant ($ US courants)`                                     | Économique             |
| `Population_urbaine_%`        | Pourcentage de la population vivant en zone urbaine            | Banque Mondiale – `Population urbaine (% du total)`                                     | Démographique          |
| `Stabilite_politique`         | Indice de stabilité politique (-2.5 à +2.5)                     | Banque Mondiale – `Indice de stabilité politique et d’absence de violence`              | Politique / Sécurité   |

### 5. PIB par habitant (Banque mondiale)

In [23]:


# Lecture du fichier PIB avec saut des lignes d’en-tête
df_pib = pd.read_csv("PIB_par_habitant.csv", skiprows=4)

# Aperçu des données
df_pib.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,PIB par habitant ($ US courants),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,27441.529662,28440.051964,30082.127645,31096.205074,22855.932320,27200.061079,30559.533535,33984.790620,NaN,NaN
1,NaN,AFE,PIB par habitant ($ US courants),NY.GDP.PCAP.CD,186.121835,186.941781,197.402402,225.440494,208.999748,226.876513,...,1329.807285,1520.212231,1538.901679,1493.817938,1344.103210,1522.393346,1628.318944,1568.159891,1673.841139,NaN
2,Afghanistan,AFG,PIB par habitant ($ US courants),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,522.082216,525.469771,491.337221,496.602504,510.787063,356.496214,357.261153,413.757895,NaN,NaN
3,NaN,AFW,PIB par habitant ($ US courants),NY.GDP.PCAP.CD,121.939925,127.454189,133.827044,139.008291,148.549379,155.565216,...,1630.039447,1574.230560,1720.140280,1798.340685,1680.039332,1765.954788,1796.668633,1599.392983,1284.154441,NaN
4,Angola,AGO,PIB par habitant ($ US courants),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1807.952941,2437.259712,2538.591391,2189.855714,1449.922867,1925.874661,2929.694455,2309.534130,2122.083690,NaN


In [24]:
# J'ai remarqué au fur et à mesure de mes jointures qui suivent que les noms de pays ne correspondent pas tous entre les fichiers de la FAO et de la banque mondiale.
# Je vais donc créer une table de correspondance pour les faire matcher car il y a des pays importants dans la liste.

# Liste des noms de pays utilisés dans le DataFrame principal
noms_pays_df = df_final['Pays'].unique()

# Liste des noms de pays présents dans le fichier PIB
noms_pays_pib = df_pib['Country Name'].unique()

# Identifier ceux qui ne matchent pas (pays dans df_final mais pas dans df_pib)
pays_non_trouves = sorted(set(noms_pays_df) - set(noms_pays_pib))

print("Pays sans correspondance dans le CSV PIB :", pays_non_trouves)

Pays sans correspondance dans le CSV PIB : ['Bolivie (État plurinational de)', 'Chine - RAS de Hong-Kong', 'Chine - RAS de Macao', 'Chine, Taiwan Province de', 'Chine, continentale', 'Congo', "Iran (République islamique d')", 'Kirghizistan', "Royaume-Uni de Grande-Bretagne et d'Irlande du Nord", 'République de Corée', 'République de Moldova', 'République populaire démocratique de Corée', 'République-Unie de Tanzanie', 'Slovaquie', 'Tchéquie', 'Venezuela (République bolivarienne du)', 'Yémen', 'Égypte', "États-Unis d'Amérique"]


In [25]:
# on compare les noms de ces pays dans le df_final

sorted(df_final["Pays"].unique())

['Afghanistan',
 'Afrique du Sud',
 'Albanie',
 'Algérie',
 'Allemagne',
 'Angola',
 'Antigua-et-Barbuda',
 'Arabie saoudite',
 'Argentine',
 'Arménie',
 'Australie',
 'Autriche',
 'Azerbaïdjan',
 'Bahamas',
 'Bangladesh',
 'Barbade',
 'Belgique',
 'Belize',
 'Bolivie (État plurinational de)',
 'Bosnie-Herzégovine',
 'Botswana',
 'Brésil',
 'Bulgarie',
 'Burkina Faso',
 'Bélarus',
 'Bénin',
 'Cabo Verde',
 'Cambodge',
 'Cameroun',
 'Canada',
 'Chili',
 'Chine - RAS de Hong-Kong',
 'Chine - RAS de Macao',
 'Chine, Taiwan Province de',
 'Chine, continentale',
 'Chypre',
 'Colombie',
 'Congo',
 'Costa Rica',
 'Croatie',
 'Cuba',
 "Côte d'Ivoire",
 'Danemark',
 'Djibouti',
 'Dominique',
 'El Salvador',
 'Espagne',
 'Estonie',
 'Eswatini',
 'Fidji',
 'Finlande',
 'France',
 'Fédération de Russie',
 'Gabon',
 'Gambie',
 'Ghana',
 'Grenade',
 'Grèce',
 'Guatemala',
 'Guinée',
 'Guinée-Bissau',
 'Guyana',
 'Géorgie',
 'Haïti',
 'Honduras',
 'Hongrie',
 'Inde',
 'Indonésie',
 "Iran (République 

In [26]:
# Dictionnaire de correspondance entre les noms de pays dans df_final et ceux du fichier PIB
correspondance_pays_pib = {
    "Bolivie (État plurinational de)": "Bolivie",
    "Chine - RAS de Hong-Kong": "Chine, RAS de Hong Kong",
    "Chine - RAS de Macao": "Région administrative spéciale de Macao, Chine",
    "Chine, Taiwan Province de": "Taïwan",  
    "Chine, continentale": "Chine",
    "Congo": "Congo, République du",
    "Iran (République islamique d')": "Iran, République islamique d’",
    "Kirghizistan": "République kirghize",
    "Royaume-Uni de Grande-Bretagne et d'Irlande du Nord": "Royaume-Uni",
    "République de Corée": "Corée, République de",
    "République de Moldova": "Moldova",
    "République populaire démocratique de Corée": "Corée, République démocratique de",
    "République-Unie de Tanzanie": "Tanzanie",
    "Slovaquie": "République slovaque",
    "Tchéquie": "République tchèque",
    "Venezuela (République bolivarienne du)": "Venezuela",
    "Yémen": "Yémen, Rép. du",
    "Égypte": "Égypte, République arabe d’",
    "États-Unis d'Amérique": "États-Unis"
}

In [27]:
# On applique la correction des noms de pays avant de faire la jointure avec les données du PIB
df_final["Pays"] = df_final["Pays"].replace(correspondance_pays_pib)

In [28]:
# vérification des correspondances :
pays_pib = set(df_pib["Country Name"].unique())
pays_df = set(df_final["Pays"].unique())
pays_sans_correspondance = pays_df - pays_pib

print("Pays encore sans correspondance dans le fichier PIB :", pays_sans_correspondance)

Pays encore sans correspondance dans le fichier PIB : {'Taïwan'}


In [29]:
# Liste des pays restants sans correspondance
pays_sans_correspondance = set(df_final["Pays"].unique()) - set(df_pib["Country Name"].unique())
print("✅ Pays encore sans correspondance dans le CSV PIB :", pays_sans_correspondance)

✅ Pays encore sans correspondance dans le CSV PIB : {'Taïwan'}


In [30]:
# 🔎 On vérifie si tous les pays de df_final sont bien présents dans le fichier PIB


pays_df_final = set(df_final["Pays"].unique())
pays_df_pib = set(df_pib["Country Name"].unique())

pays_non_match = sorted(pays_df_final - pays_df_pib)

print("✅ Pays encore sans correspondance dans le CSV PIB :", pays_non_match)

✅ Pays encore sans correspondance dans le CSV PIB : ['Taïwan']


In [31]:
df_final.head()

,Pays,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers
0,Afghanistan,1.53,28.0,29.0,NaN,57.0,0.0,36296.113
1,Afrique du Sud,35.69,1667.0,514.0,63.0,2118.0,-0.0,57009.756
2,Albanie,16.36,13.0,38.0,0.0,47.0,4.0,2884.169
3,Algérie,6.38,275.0,2.0,0.0,277.0,0.0,41389.189
4,Allemagne,19.47,1514.0,842.0,646.0,1739.0,-29.0,82658.409


In [32]:
# Ne garder que les colonnes Pays et 2017
df_pib_2017 = df_pib[['Country Name', '2017']].copy()

# Renommer les colonnes pour correspondre à df_merge
df_pib_2017.rename(columns={'Country Name': 'Pays', '2017': 'PIB_hab_USD'}, inplace=True)

# Suppression des pays avec valeurs manquantes
df_pib_2017.dropna(subset=['PIB_hab_USD'], inplace=True)

# verif
df_pib_2017.head()

,Pays,PIB_hab_USD
0,Aruba,28440.051964
1,NaN,1520.212231
2,Afghanistan,525.469771
3,NaN,1574.230560
4,Angola,2437.259712


In [33]:
# Suppression des lignes sans nom de pays
df_pib_2017 = df_pib_2017[df_pib_2017['Pays'].notna()].reset_index(drop=True)

# Vérification
df_pib_2017.head()

,Pays,PIB_hab_USD
0,Aruba,28440.051964
1,Afghanistan,525.469771
2,Angola,2437.259712
3,Albanie,4614.047969
4,Andorre,40672.994335


In [34]:
df_pib_2017.describe()

,PIB_hab_USD
count,256.000000
mean,16719.816027
std,24505.206102
min,236.694473
25%,2188.606539
50%,6446.312069
75%,20866.941170
max,170663.375248


### 5.1 – Fusion avec le PIB par habitant

Nous joignons les données de PIB par habitant à notre DataFrame principal, en effectuant une jointure `left` pour ne conserver que les pays déjà sélectionnés dans notre base consolidée.  
Cela permet de ne pas réintroduire des pays hors périmètre que nous n'avons pas retenus dans les étapes précédentes.

In [36]:
# On merge en utilisant les noms corrigés
df_final = df_final.merge(df_pib_2017, how="left", left_on="Pays", right_on="Pays")

df_final.head()

,Pays,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers,PIB_hab_USD
0,Afghanistan,1.53,28.0,29.0,NaN,57.0,0.0,36296.113,525.469771
1,Afrique du Sud,35.69,1667.0,514.0,63.0,2118.0,-0.0,57009.756,6618.335083
2,Albanie,16.36,13.0,38.0,0.0,47.0,4.0,2884.169,4614.047969
3,Algérie,6.38,275.0,2.0,0.0,277.0,0.0,41389.189,4554.667540
4,Allemagne,19.47,1514.0,842.0,646.0,1739.0,-29.0,82658.409,45526.599958


### 6. Population urbaine (% du total)

In [38]:
# Lecture du fichier en ignorant les 2 premières lignes inutiles
df_urbain = pd.read_csv("population_urbaine.csv", sep=',', skiprows=4)

df_urbain.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,Population urbaine (% du total),SP.URB.TOTL.IN.ZS,50.776000,50.761000,50.746000,50.730000,50.715000,50.700000,...,43.192000,43.293000,43.411000,43.546000,43.697000,43.866000,44.052000,44.254000,44.474000,NaN
1,NaN,AFE,Population urbaine (% du total),SP.URB.TOTL.IN.ZS,14.577252,14.825175,15.083802,15.363045,15.655383,15.955912,...,34.919544,35.396289,35.893398,36.384272,36.884034,37.393633,37.909012,38.424898,38.949114,NaN
2,Afghanistan,AFG,Population urbaine (% du total),SP.URB.TOTL.IN.ZS,8.401000,8.684000,8.976000,9.276000,9.586000,9.904000,...,25.020000,25.250000,25.495000,25.754000,26.026000,26.314000,26.616000,26.933000,27.265000,NaN
3,NaN,AFW,Population urbaine (% du total),SP.URB.TOTL.IN.ZS,14.710006,15.094445,15.487932,15.900682,16.331319,16.779793,...,45.473850,46.094137,46.709753,47.322617,47.931021,48.531971,49.129808,49.711184,50.280934,NaN
4,Angola,AGO,Population urbaine (% du total),SP.URB.TOTL.IN.ZS,10.435000,10.798000,11.204000,11.624000,12.058000,12.504000,...,64.149000,64.839000,65.514000,66.177000,66.825000,67.460000,68.081000,68.688000,69.281000,NaN


In [39]:
# On isole les colonnes utiles
df_urbain_2017 = df_urbain[['Country Name', '2017']].copy()

# On renomme pour harmoniser avec les autres DataFrames
df_urbain_2017.columns = ['Pays', 'Population_urbaine_%']

df_urbain_2017.head()

,Pays,Population_urbaine_%
0,Aruba,43.293000
1,NaN,35.396289
2,Afghanistan,25.250000
3,NaN,46.094137
4,Angola,64.839000


In [40]:
# On enlève les lignes sans pays (ex : AFE, AFW...)
df_urbain_2017 = df_urbain_2017.dropna(subset=['Pays'])

In [41]:
# Jointure externe avec indicateur de correspondance
df_final = pd.merge(df_final, df_urbain_2017, on='Pays', how='left', indicator='source_jointure_urbain')

# Affichage des correspondances
print(df_final['source_jointure_urbain'].value_counts())

source_jointure_urbain
both          171
left_only       1
right_only      0
Name: count, dtype: int64


In [42]:
df_final.head()

,Pays,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers,PIB_hab_USD,Population_urbaine_%,source_jointure_urbain
0,Afghanistan,1.53,28.0,29.0,NaN,57.0,0.0,36296.113,525.469771,25.250,both
1,Afrique du Sud,35.69,1667.0,514.0,63.0,2118.0,-0.0,57009.756,6618.335083,65.850,both
2,Albanie,16.36,13.0,38.0,0.0,47.0,4.0,2884.169,4614.047969,59.383,both
3,Algérie,6.38,275.0,2.0,0.0,277.0,0.0,41389.189,4554.667540,72.052,both
4,Allemagne,19.47,1514.0,842.0,646.0,1739.0,-29.0,82658.409,45526.599958,77.261,both


### 7. Stabilité politique (Banque mondiale)

In [44]:
# 1. Lecture du fichier avec skiprows pour ignorer les lignes d’en-tête inutiles
df_stab = pd.read_csv("stabilite_politique.csv", skiprows=3)

# 2. Sélection de la colonne 2017
df_stab_2017 = df_stab[["Country Name", "2017"]].copy()

# 3. Renommage des colonnes
df_stab_2017.columns = ["Pays", "Stabilite_politique"]

# 4. Suppression des lignes sans nom de pays
df_stab_2017 = df_stab_2017.dropna(subset=["Pays"])

df_stab_2017.head()

,Pays,Stabilite_politique
0,Aruba,1.313846
2,Afghanistan,-2.794976
4,Angola,-0.387895
5,Albanie,0.373771
6,Andorre,1.392890


In [45]:
# Jointure avec indicateur
df_merged = pd.merge(df_final, df_stab_2017, on="Pays", how="left", indicator="source_jointure_stab")

# Affichage du résultat
print(df_merged["source_jointure_stab"].value_counts())

source_jointure_stab
both          171
left_only       1
right_only      0
Name: count, dtype: int64


### 8. Finalisation du dataframe

In [47]:
# On garde la version propre
df_final = df_merged.drop(columns=["source_jointure_stab","source_jointure_urbain"])

df_final.head()

,Pays,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers,PIB_hab_USD,Population_urbaine_%,Stabilite_politique
0,Afghanistan,1.53,28.0,29.0,NaN,57.0,0.0,36296.113,525.469771,25.250,-2.794976
1,Afrique du Sud,35.69,1667.0,514.0,63.0,2118.0,-0.0,57009.756,6618.335083,65.850,-0.284804
2,Albanie,16.36,13.0,38.0,0.0,47.0,4.0,2884.169,4614.047969,59.383,0.373771
3,Algérie,6.38,275.0,2.0,0.0,277.0,0.0,41389.189,4554.667540,72.052,-0.919614
4,Allemagne,19.47,1514.0,842.0,646.0,1739.0,-29.0,82658.409,45526.599958,77.261,0.574381


In [48]:
df_final.describe()


,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers,PIB_hab_USD,Population_urbaine_%,Stabilite_politique
count,172.000000,168.000000,170.000000,135.000000,170.000000,169.000000,1.720000e+02,169.000000,171.000000,169.000000
mean,20.213372,725.190476,89.529412,132.185185,687.594118,13.668639,4.284175e+04,14214.053306,58.801737,-0.076988
std,15.860311,2501.457125,186.669983,513.784440,2187.184747,75.364884,1.530637e+05,19310.308470,22.072448,0.891267
min,0.130000,0.000000,0.000000,0.000000,2.000000,-119.000000,5.204500e+01,432.324026,16.350000,-2.934317
25%,6.440000,13.750000,3.000000,0.000000,30.500000,0.000000,2.874480e+03,2012.286626,41.367000,-0.629498
50%,18.090000,70.000000,16.000000,3.000000,100.000000,0.000000,9.757833e+03,6049.521286,59.383000,-0.039427
75%,30.037500,409.750000,81.250000,32.000000,368.250000,7.000000,3.013874e+04,17645.529872,77.119000,0.628550
max,72.310000,21914.000000,1069.000000,4223.000000,18266.000000,859.000000,1.421022e+06,110193.213797,100.000000,1.561946


In [49]:
df_final.isna().sum()

Pays                                0
conso_volaille_kg_hab               0
prod_volaille_milliers_tonnes       4
import_volaille_milliers_tonnes     2
export_volaille_milliers_tonnes    37
dispo_volaille_milliers_tonnes      2
variation_stock_milliers_tonnes     3
Population_milliers                 0
PIB_hab_USD                         3
Population_urbaine_%                1
Stabilite_politique                 3
dtype: int64

In [50]:
# Afficher les pays pour lesquels les exportations de volaille sont manquantes
# Version avec plus d'infos pour mieux analyser les cas
df_final[df_final['export_volaille_milliers_tonnes'].isna()][[
    'Pays',
    'prod_volaille_milliers_tonnes',
    'import_volaille_milliers_tonnes',
    'variation_stock_milliers_tonnes',
    'dispo_volaille_milliers_tonnes'
]]

,Pays,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,variation_stock_milliers_tonnes,dispo_volaille_milliers_tonnes
0,Afghanistan,28.0,29.0,0.0,57.0
13,Bahamas,6.0,24.0,4.0,26.0
14,Bangladesh,249.0,0.0,-0.0,250.0
25,Burkina Faso,46.0,0.0,-0.0,46.0
26,Cabo Verde,1.0,12.0,4.0,10.0
27,Cambodge,28.0,10.0,-0.0,38.0
41,Cuba,29.0,312.0,-1.0,342.0
43,Djibouti,NaN,3.0,0.0,3.0
53,Éthiopie,14.0,1.0,0.0,14.0
59,Gambie,2.0,16.0,10.0,8.0


## Estimation des exportations manquantes (formule simplifiée)

Pour certains pays, les données d’exportation de viande de volaille sont manquantes dans le fichier d’origine.

Nous allons proposer une **formule comptable simplifiée** pour estimer les exportations manquantes :

\[
\Export estimée = Production + Importations + Variation de stock - Disponibilité intérieure
\]

📌 **Attention :** cette formule ne permet pas de retrouver avec précision les valeurs d’exportation réelles pour de nombreux pays. Nous l'utilisons uniquement à des fins de **remplissage ponctuel** là où la donnée officielle est absente (`NaN`).

In [69]:
# Étape 1 – Calcul de l'export estimée avec la formule simplifiée
# (on calcule pour tous les pays, pas seulement ceux avec valeur manquante, pour pouvoir vérifier ensuite)

df_final['export_calculee_simplifiee'] = (
    df_final['prod_volaille_milliers_tonnes']
    + df_final['import_volaille_milliers_tonnes']
    + df_final['variation_stock_milliers_tonnes']
    - df_final['dispo_volaille_milliers_tonnes']
)

# Étape 2 – Comparaison avec les valeurs réelles (quand elles existent)
# Cela nous permettra de voir si la formule colle bien ou non aux données FAO

df_final['ecart_export'] = df_final['export_volaille_milliers_tonnes'] - df_final['export_calculee_simplifiee']

# Étape 3 – Liste des pays où l'écart est supérieur à 100 milliers de tonnes
# (cette vérification montre que la formule est très imprécise)

grands_ecarts = df_final[df_final['ecart_export'].abs() > 100][[
    'Pays',
    'export_volaille_milliers_tonnes',
    'export_calculee_simplifiee',
    'ecart_export',
    'prod_volaille_milliers_tonnes',
    'import_volaille_milliers_tonnes',
    'variation_stock_milliers_tonnes',
    'dispo_volaille_milliers_tonnes'
]].sort_values(by='ecart_export', key=abs, ascending=False)

# Affichage des plus gros écarts entre la valeur officielle et la valeur estimée
grands_ecarts.head(150)

,Pays,export_volaille_milliers_tonnes,export_calculee_simplifiee,ecart_export,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,variation_stock_milliers_tonnes,dispo_volaille_milliers_tonnes
81,Japon,10.0,1728.0,-1718.0,2215.0,1069.0,859.0,2415.0
126,Pologne,1025.0,1475.0,-450.0,2351.0,55.0,225.0,1156.0
57,France,501.0,866.0,-365.0,1750.0,506.0,183.0,1573.0
96,Malaisie,44.0,299.0,-255.0,1724.0,68.0,128.0,1621.0
72,Inde,4.0,-235.0,239.0,3545.0,0.0,-119.0,3661.0
62,Grèce,29.0,265.0,-236.0,246.0,79.0,118.0,178.0
130,"Corée, République de",6.0,236.0,-230.0,838.0,137.0,115.0,854.0
7,Arabie saoudite,10.0,-205.0,215.0,616.0,722.0,-108.0,1435.0
20,Bolivie,1.0,208.0,-207.0,533.0,1.0,103.0,429.0
163,Turquie,429.0,613.0,-184.0,2192.0,3.0,92.0,1674.0


Certains pays ont des écarts importants par rapport aux exports calculés, cela démontre que la formule n'est pas parfaite. Néanmoins cela ne concerne que quelques pays sur l'échantillon et cela semble acceptable pour conserver un ensemble de données le plus large possible.

In [53]:
# Étape 4 – Utilisation de la formule uniquement pour combler les valeurs manquantes

df_final['export_volaille_milliers_tonnes'] = df_final.apply(
    lambda x: x['export_calculee_simplifiee'] if pd.isnull(x['export_volaille_milliers_tonnes']) else x['export_volaille_milliers_tonnes'],
    axis=1
)

df_final.head()

,Pays,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers,PIB_hab_USD,Population_urbaine_%,Stabilite_politique,export_calculee_simplifiee,ecart_export
0,Afghanistan,1.53,28.0,29.0,0.0,57.0,0.0,36296.113,525.469771,25.250,-2.794976,0.0,NaN
1,Afrique du Sud,35.69,1667.0,514.0,63.0,2118.0,-0.0,57009.756,6618.335083,65.850,-0.284804,63.0,0.0
2,Albanie,16.36,13.0,38.0,0.0,47.0,4.0,2884.169,4614.047969,59.383,0.373771,8.0,-8.0
3,Algérie,6.38,275.0,2.0,0.0,277.0,0.0,41389.189,4554.667540,72.052,-0.919614,0.0,0.0
4,Allemagne,19.47,1514.0,842.0,646.0,1739.0,-29.0,82658.409,45526.599958,77.261,0.574381,588.0,58.0


In [54]:
df_final['export_volaille_milliers_tonnes'].describe()

count     168.000000
mean      107.000000
std       463.087969
min        -7.000000
25%         0.000000
50%         1.500000
75%        17.750000
max      4223.000000
Name: export_volaille_milliers_tonnes, dtype: float64

In [55]:
df_final[df_final['export_volaille_milliers_tonnes'] < 0]

,Pays,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers,PIB_hab_USD,Population_urbaine_%,Stabilite_politique,export_calculee_simplifiee,ecart_export
14,Bangladesh,1.50,249.0,0.0,-1.0,250.0,-0.0,159685.424,1811.082217,35.858,-1.253627,-1.0,NaN
41,Cuba,23.72,29.0,312.0,-2.0,342.0,-1.0,11339.254,8610.608295,76.977,0.638511,-2.0,NaN
95,Madagascar,2.87,81.0,0.0,-1.0,82.0,-0.0,25570.512,491.866849,36.522,-0.302837,-1.0,NaN
99,Mali,2.83,48.0,1.0,-7.0,52.0,-4.0,18512.430,924.057532,41.572,-1.903492,-7.0,NaN
107,Mozambique,3.59,92.0,24.0,-1.0,116.0,-1.0,28649.018,470.938563,35.455,-0.918099,-1.0,NaN
115,Nouvelle-Calédonie,38.71,1.0,9.0,-2.0,11.0,-1.0,277.150,32354.710283,70.256,NaN,-2.0,NaN
135,Tanzanie,1.88,105.0,2.0,-1.0,108.0,-0.0,54660.339,986.674011,33.053,-0.570162,-1.0,NaN
171,Zimbabwe,4.68,69.0,6.0,-1.0,76.0,-0.0,14236.595,3448.086991,32.237,-0.710431,-1.0,NaN


In [56]:
# Certaines exportations négatives sont probablement des valeurs manquantes mal codées.
# On les remplace ici par 0 car il est économiquement peu probable qu'un pays exporte une quantité négative.
df_final.loc[df_final['export_volaille_milliers_tonnes'] < 0, 'export_volaille_milliers_tonnes'] = 0

In [57]:
df_final['export_volaille_milliers_tonnes'].describe()

count     168.000000
mean      107.095238
std       463.065421
min         0.000000
25%         0.000000
50%         1.500000
75%        17.750000
max      4223.000000
Name: export_volaille_milliers_tonnes, dtype: float64

In [58]:
df_final.describe()

,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers,PIB_hab_USD,Population_urbaine_%,Stabilite_politique,export_calculee_simplifiee,ecart_export
count,172.000000,168.000000,170.000000,168.000000,170.000000,169.000000,1.720000e+02,169.000000,171.000000,169.000000,167.000000,134.000000
mean,20.213372,725.190476,89.529412,107.095238,687.594118,13.668639,4.284175e+04,14214.053306,58.801737,-0.076988,134.544910,-33.537313
std,15.860311,2501.457125,186.669983,463.065421,2187.184747,75.364884,1.530637e+05,19310.308470,22.072448,0.891267,496.492052,168.724746
min,0.130000,0.000000,0.000000,0.000000,2.000000,-119.000000,5.204500e+01,432.324026,16.350000,-2.934317,-235.000000,-1718.000000
25%,6.440000,13.750000,3.000000,0.000000,30.500000,0.000000,2.874480e+03,2012.286626,41.367000,-0.629498,0.000000,-22.000000
50%,18.090000,70.000000,16.000000,1.500000,100.000000,0.000000,9.757833e+03,6049.521286,59.383000,-0.039427,3.000000,-1.000000
75%,30.037500,409.750000,81.250000,17.750000,368.250000,7.000000,3.013874e+04,17645.529872,77.119000,0.628550,51.000000,1.000000
max,72.310000,21914.000000,1069.000000,4223.000000,18266.000000,859.000000,1.421022e+06,110193.213797,100.000000,1.561946,4222.000000,239.000000


In [59]:
df_final.isna().sum().sort_values(ascending=False)

ecart_export                       38
export_calculee_simplifiee          5
prod_volaille_milliers_tonnes       4
export_volaille_milliers_tonnes     4
variation_stock_milliers_tonnes     3
PIB_hab_USD                         3
Stabilite_politique                 3
import_volaille_milliers_tonnes     2
dispo_volaille_milliers_tonnes      2
Population_urbaine_%                1
Pays                                0
conso_volaille_kg_hab               0
Population_milliers                 0
dtype: int64

In [60]:
colonnes_critiques = [
     'prod_volaille_milliers_tonnes',
    'import_volaille_milliers_tonnes',
    'dispo_volaille_milliers_tonnes',
    'PIB_hab_USD',
    'Population_urbaine_%',
    'Stabilite_politique'
]

df_nan_critiques = df_final[df_final[colonnes_critiques].isna().any(axis=1)]
print(f"Nombre de pays avec NaN sur les colonnes critiques : {df_nan_critiques.shape[0]}")

df_nan_critiques.head(25)

Nombre de pays avec NaN sur les colonnes critiques : 9


,Pays,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers,PIB_hab_USD,Population_urbaine_%,Stabilite_politique,export_calculee_simplifiee,ecart_export
34,Taïwan,33.17,652.0,161.0,8.0,785.0,20.0,23674.546,NaN,NaN,NaN,48.0,-40.0
43,Djibouti,2.68,NaN,3.0,NaN,3.0,0.0,944.099,2618.955211,77.648,-0.733945,NaN,NaN
98,Maldives,13.50,NaN,12.0,NaN,12.0,0.0,496.402,10509.628806,39.380,0.224532,NaN,NaN
115,Nouvelle-Calédonie,38.71,1.0,9.0,0.0,11.0,-1.0,277.150,32354.710283,70.256,NaN,-2.0,NaN
119,Ouzbékistan,1.96,NaN,NaN,NaN,NaN,NaN,31959.785,2190.604116,50.550,-0.244979,NaN,NaN
127,Polynésie française,47.40,1.0,15.0,0.0,15.0,0.0,276.102,20826.931249,61.784,NaN,1.0,-1.0
132,République démocratique populaire lao,10.91,NaN,NaN,NaN,NaN,NaN,6953.035,2432.430595,34.368,0.394013,NaN,NaN
134,"Corée, République démocratique de",1.62,43.0,0.0,0.0,43.0,-0.0,25429.825,NaN,61.678,-0.500023,0.0,0.0
167,Venezuela,20.28,600.0,25.0,0.0,626.0,-1.0,29402.484,NaN,88.183,-1.252241,-2.0,2.0


In [62]:
# Suppression des pays dont les données critiques (production, importation, disponibilité) sont manquantes
colonnes_critiques = [
    'prod_volaille_milliers_tonnes',
    'import_volaille_milliers_tonnes',
    'dispo_volaille_milliers_tonnes',
    'PIB_hab_USD',
    'Population_urbaine_%',
    'Stabilite_politique'



]

df_final = df_final[~df_final[colonnes_critiques].isna().any(axis=1)]

df_final.head()

,Pays,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers,PIB_hab_USD,Population_urbaine_%,Stabilite_politique,export_calculee_simplifiee,ecart_export
0,Afghanistan,1.53,28.0,29.0,0.0,57.0,0.0,36296.113,525.469771,25.250,-2.794976,0.0,NaN
1,Afrique du Sud,35.69,1667.0,514.0,63.0,2118.0,-0.0,57009.756,6618.335083,65.850,-0.284804,63.0,0.0
2,Albanie,16.36,13.0,38.0,0.0,47.0,4.0,2884.169,4614.047969,59.383,0.373771,8.0,-8.0
3,Algérie,6.38,275.0,2.0,0.0,277.0,0.0,41389.189,4554.667540,72.052,-0.919614,0.0,0.0
4,Allemagne,19.47,1514.0,842.0,646.0,1739.0,-29.0,82658.409,45526.599958,77.261,0.574381,588.0,58.0


In [63]:
df_final.isna().sum().sort_values(ascending=False)

ecart_export                       33
variation_stock_milliers_tonnes     1
export_calculee_simplifiee          1
Pays                                0
conso_volaille_kg_hab               0
prod_volaille_milliers_tonnes       0
import_volaille_milliers_tonnes     0
export_volaille_milliers_tonnes     0
dispo_volaille_milliers_tonnes      0
Population_milliers                 0
PIB_hab_USD                         0
Population_urbaine_%                0
Stabilite_politique                 0
dtype: int64

In [79]:
# Affichage des pays avec une valeur manquante dans "variation_stock_milliers_tonnes"
df_final[df_final["variation_stock_milliers_tonnes"].isna()]

,Pays,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers,PIB_hab_USD,Population_urbaine_%,Stabilite_politique,export_calculee_simplifiee,ecart_export
124,Pérou,13.47,1465.0,60.0,1.0,1523.0,NaN,31444.298,6736.166937,77.72,-0.26351,NaN,NaN


In [83]:
# Affichage des pays avec une valeur manquante dans "export_calculee_simplifiee"
df_final[df_final["export_calculee_simplifiee"].isna()]

,Pays,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers,PIB_hab_USD,Population_urbaine_%,Stabilite_politique,export_calculee_simplifiee,ecart_export
124,Pérou,13.47,1465.0,60.0,1.0,1523.0,NaN,31444.298,6736.166937,77.72,-0.26351,NaN,NaN


In [85]:
# Affichage des pays ayant une valeur manquante dans les deux colonnes
df_final[df_final["variation_stock_milliers_tonnes"].isna() & df_final["export_calculee_simplifiee"].isna()]

,Pays,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers,PIB_hab_USD,Population_urbaine_%,Stabilite_politique,export_calculee_simplifiee,ecart_export
124,Pérou,13.47,1465.0,60.0,1.0,1523.0,NaN,31444.298,6736.166937,77.72,-0.26351,NaN,NaN


In [87]:
# Remplacement manuel de la valeur manquante pour le Pérou
df_final.loc[df_final["Pays"] == "Pérou", "variation_stock_milliers_tonnes"] = 0.0

# Recalcul de la colonne "export_calculee_simplifiee"
df_final["export_calculee_simplifiee"] = (
    df_final["prod_volaille_milliers_tonnes"]
    + df_final["import_volaille_milliers_tonnes"]
    - df_final["dispo_volaille_milliers_tonnes"]
    - df_final["variation_stock_milliers_tonnes"]
)

In [93]:
# Vérification des valeurs manquantes dans la colonne export_calculee_simplifiee
df_final[df_final["export_calculee_simplifiee"].isna()]

df_final.describe()

,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers,PIB_hab_USD,Population_urbaine_%,Stabilite_politique,export_calculee_simplifiee,ecart_export
count,163.000000,163.000000,163.000000,163.000000,163.000000,163.00000,1.630000e+02,163.000000,163.000000,163.000000,163.000000,162.000000
mean,20.285092,739.478528,91.993865,110.331288,707.950920,14.06135,4.447465e+04,14302.096616,58.719325,-0.066861,109.460123,-27.413580
std,15.840069,2537.859578,189.949717,469.780196,2230.715358,76.70536,1.570643e+05,19537.802878,22.279453,0.899597,469.846918,153.907193
min,0.130000,0.000000,0.000000,0.000000,2.000000,-119.00000,5.204500e+01,432.324026,16.350000,-2.934317,-1.000000,-1718.000000
25%,6.760000,14.500000,3.000000,0.000000,32.500000,-0.00000,2.932820e+03,1957.201993,41.367000,-0.627854,0.000000,-12.250000
50%,18.200000,71.000000,16.000000,2.000000,108.000000,0.00000,9.785843e+03,6049.521286,58.299000,-0.002855,1.000000,0.000000
75%,29.640000,376.500000,83.000000,20.000000,364.500000,7.50000,3.127447e+04,17612.085824,77.119000,0.640441,17.500000,0.750000
max,72.310000,21914.000000,1069.000000,4223.000000,18266.000000,859.00000,1.421022e+06,110193.213797,100.000000,1.561946,4222.000000,239.000000


Nous n'avons plus de NaN dans les pays conservés. Il nous reste 163 pays !

In [96]:
# Suppression des colonnes intuiles pour alléger le dataframe.

df_final.drop(columns=[
    'export_calculee_simplifiee',
    'ecart_export'      
], inplace=True)

df_final.info()
df_final.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 163 entries, 0 to 171
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Pays                             163 non-null    object 
 1   conso_volaille_kg_hab            163 non-null    float64
 2   prod_volaille_milliers_tonnes    163 non-null    float64
 3   import_volaille_milliers_tonnes  163 non-null    float64
 4   export_volaille_milliers_tonnes  163 non-null    float64
 5   dispo_volaille_milliers_tonnes   163 non-null    float64
 6   variation_stock_milliers_tonnes  163 non-null    float64
 7   Population_milliers              163 non-null    float64
 8   PIB_hab_USD                      163 non-null    float64
 9   Population_urbaine_%             163 non-null    float64
 10  Stabilite_politique              163 non-null    float64
dtypes: float64(10), object(1)
memory usage: 15.3+ KB


Pays                               0
conso_volaille_kg_hab              0
prod_volaille_milliers_tonnes      0
import_volaille_milliers_tonnes    0
export_volaille_milliers_tonnes    0
dispo_volaille_milliers_tonnes     0
variation_stock_milliers_tonnes    0
Population_milliers                0
PIB_hab_USD                        0
Population_urbaine_%               0
Stabilite_politique                0
dtype: int64

## 🧮 Variables construites à partir des données disponibles

Pour enrichir l’analyse et anticiper les étapes de réduction de dimension (ACP), trois variables supplémentaires ont été calculées :

1. **PIB_total_milliards_USD** : estimation du PIB total d’un pays  
   *Formule* : `(PIB par habitant × Population) / 1 000 000`

2. **autosuffisance_volaille** : indicateur de dépendance ou d’indépendance alimentaire sur la volaille  
   *Formule* : `Production de volaille / Disponibilité totale`

3. **Population_urbaine_millions** : estimation du nombre d’habitants en milieu urbain  
   *Formule* : `(Population × % urbaine) / 100 000`

Ces variables permettent d’ajouter des perspectives économiques, alimentaires et démographiques à l’analyse multivariée.

In [100]:
# Création de variables construites à partir des colonnes existantes

# 1. PIB total en milliards de dollars
df_final['PIB_total_milliards_USD'] = (df_final['PIB_hab_USD'] * df_final['Population_milliers']) / 1_000_000

# 2. Taux d'autosuffisance en volaille (production / consommation estimée)
df_final['autosuffisance_volaille'] = df_final['prod_volaille_milliers_tonnes'] / df_final['dispo_volaille_milliers_tonnes']

# 3. Population urbaine en millions
df_final['Population_urbaine_millions'] = (df_final['Population_milliers'] * df_final['Population_urbaine_%']) / 100_000

In [102]:
df_final.describe()

,conso_volaille_kg_hab,prod_volaille_milliers_tonnes,import_volaille_milliers_tonnes,export_volaille_milliers_tonnes,dispo_volaille_milliers_tonnes,variation_stock_milliers_tonnes,Population_milliers,PIB_hab_USD,Population_urbaine_%,Stabilite_politique,PIB_total_milliards_USD,autosuffisance_volaille,Population_urbaine_millions
count,163.000000,163.000000,163.000000,163.000000,163.000000,163.00000,1.630000e+02,163.000000,163.000000,163.000000,163.000000,163.000000,163.000000
mean,20.285092,739.478528,91.993865,110.331288,707.950920,14.06135,4.447465e+04,14302.096616,58.719325,-0.066861,490.982627,0.786071,24.389437
std,15.840069,2537.859578,189.949717,469.780196,2230.715358,76.70536,1.570643e+05,19537.802878,22.279453,0.899597,1912.552920,0.491137,78.830563
min,0.130000,0.000000,0.000000,0.000000,2.000000,-119.00000,5.204500e+01,432.324026,16.350000,-2.934317,0.211541,0.000000,0.016016
25%,6.760000,14.500000,3.000000,0.000000,32.500000,-0.00000,2.932820e+03,1957.201993,41.367000,-0.627854,12.488096,0.400000,1.782438
50%,18.200000,71.000000,16.000000,2.000000,108.000000,0.00000,9.785843e+03,6049.521286,58.299000,-0.002855,48.377919,0.882353,5.303324
75%,29.640000,376.500000,83.000000,20.000000,364.500000,7.50000,3.127447e+04,17612.085824,77.119000,0.640441,277.244179,1.000000,16.147559
max,72.310000,21914.000000,1069.000000,4223.000000,18266.000000,859.00000,1.421022e+06,110193.213797,100.000000,1.561946,19520.598103,3.046053,823.624230


## 🔄 Résumé des étapes de préparation des données

- Importation et nettoyage des différentes sources de données (volaille, PIB, population, stabilité politique...).
- Harmonisation des noms de pays entre les différentes sources (jointures manuelles pour les cas ambigus).
- Gestion des valeurs manquantes :
  - Remplissage des valeurs manquantes pour l’exportation de volaille via une formule simplifiée.
  - Suppression des pays avec des données critiques manquantes si non récupérables.
- Finalisation d’un DataFrame propre comprenant **163 pays** avec **0 valeur manquante** sur les colonnes clés.

In [108]:
# Export du DataFrame final nettoyé pour l'utiliser dans le notebook d'exploration
df_final.to_csv("df_final.csv", index=False)

---

## ✅ Conclusion – Notebook de préparation

Dans ce notebook, nous avons :

- Importé et fusionné plusieurs sources de données
- Harmonisé les noms de pays pour garantir des jointures correctes
- Nettoyé les valeurs manquantes et incohérentes
- Corrigé les données d'exportation de volaille à l'aide d'une formule de calcul
- Créé trois variables supplémentaires utiles à l’analyse
- Construit un DataFrame propre, cohérent et prêt à être exploré

➡️ Le fichier `df_final.csv` est maintenant exporté et servira de base pour le second notebook consacré à l'exploration des données et à l'analyse multivariée (ACP, clustering).

---